<CENTER>
    <h1>LegalTech HackZurich 2020</h1>
    <br><hr></br>
    <a href="http://universidad.ch/hackzurich/suite/" class="icons"><img src="https://hackzurich.com/media/pages/workshops/15-legaltech/4173366116-1599065131/legaltech-600x-copy-600x.png" style="width:20%"></a>
</CENTER>

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [ ]:
from IPython.display import IFrame
IFrame('./contract-page-example.png', width="100%", height=800)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract

In [ ]:
import imutils
from imutils import contours
from imutils.perspective import four_point_transform
from skimage.filters import threshold_local

In [ ]:
def displayColorImage(image, title):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), interpolation="nearest")
    plt.title(title)
    plt.show()

In [ ]:
def displayGrayScaleImage(image, title):
    plt.imshow(image, cmap='gray', vmin = 0.0, vmax = 255.0)
    plt.title(title)
    plt.show()

In [ ]:
## clean ol files
import os
os.system('rm -r boxes.png numbers.png')

In [ ]:
#orig = cv2.imread("receipt05.jpg")
#orig = cv2.imread("IMG_20200908_181359.jpg")
orig = cv2.imread("contract-page-example.png")

displayColorImage(orig, "original")

In [ ]:
image = orig.copy()
ratio = image.shape[0]/500.0
image = imutils.resize(image, height=500)
displayColorImage(image,"resized")

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
displayGrayScaleImage(gray,"grayscale image")
gray = cv2.GaussianBlur(gray, (5,5),0)
displayGrayScaleImage(gray,"after removing noise")
edged = cv2.Canny(gray, 75, 200)
displayGrayScaleImage(edged,"edges")

In [ ]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:4]

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)
    if len(approx) == 4 :
        screenCnt = approx
        break

cv2.drawContours(image, [screenCnt], -1, (0,255,0),2)
displayColorImage(image, "with contours")
print(screenCnt)
print(screenCnt.reshape(4,2))

In [ ]:
warped = four_point_transform(orig, screenCnt.reshape(4,2) * ratio)
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
displayGrayScaleImage(warped, "warped grayscale")

In [ ]:
cv2.imwrite('DR.png', warped)
overlay = warped.copy()

In [ ]:
#warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35, 7)
#displayGrayScaleImage(warped, "warped post adpative threshold")

In [ ]:
##img = cv2.cvtColor(warped,cv2.COLOR_BGR2RGB)

In [ ]:
##cv2.imshow('Result',img)

In [ ]:
print(pytesseract.image_to_string(warped))

In [ ]:
rst = pytesseract.image_to_string(warped)
f = open("result.txt", "w")
f.write(rst)
f.close()

In [ ]:
import time

In [ ]:
img = warped
#############################################
#### Detecting Words  ######
#############################################
#[   0          1           2           3           4          5         6       7       8        9        10       11 ]
#['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text']
sortfile = 'names-sorted-uniq.txt' ## file coming from Notebook name-extract-00.ipynb 

boxes = pytesseract.image_to_data(img)
for a,b in enumerate(boxes.splitlines()):
        print(b)
        if a!=0:
            b = b.split()
            if len(b)==12:
                if b[11] in open(sortfile).read():
                    x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                    #cv2.putText(img,b[11],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)
                    cv2.rectangle(img, (x,y), (x+w, y+h), (50, 50, 255), 18)

In [ ]:
#displayColorImage(img, "original")

In [ ]:
cv2.imwrite('boxes.png', img)

In [ ]:
##############################################
##### Detecting ONLY Digits  ######
##############################################
#hImg, wImg = overlay.shape
#conf = r'--oem 3 --psm 6 outputbase digits'
#boxes = pytesseract.image_to_boxes(overlay,config=conf)
#for b in boxes.splitlines():
#    print(b)
#    b = b.split(' ')
#    print(b)
#    x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
#    #cv2.rectangle(img, (x,hImg- y), (w,hImg- h), (50, 50, 255), 2)
#    cv2.putText(overlay,b[0],(x,hImg- y+20),cv2.FONT_HERSHEY_SIMPLEX,1,(50,50,255),2)

In [ ]:
#cv2.imwrite('numbers.png', overlay)

## New picture

In [ ]:
import imageio
import matplotlib.pyplot as plt
%matplotlib inline
picBox = imageio.imread('boxes.png')
plt.figure(figsize = (25,25))
plt.imshow(picBox)

In [ ]:
#picNum = imageio.imread('numbers.png')
#plt.figure(figsize = (25,25))
#plt.imshow(picNum)

In [ ]:
from IPython.display import IFrame
IFrame('./boxes.png', width="100%", height=800)

<CENTER>
    <a href="http://universidad.ch/hackzurich/suite/" class="icons"><img src="http://universidad.ch/hackzurich/suite/logos/footer-hackathon.png" style="width:90%"></a>
</CENTER>